In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
import numpy as np

In [14]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.


True

In [15]:
wandb.init(
    project="Experiments Series 1.01",
    config={
        "Layer1": 256,
        "Activation_1": "relu",
        "Dropout1": "No dropout",
        "Layer2": 256,
        "Activation_2": "sigmoid",
        "Dropout2": "No dropout",
        "Layer3": 10,
        "Activation_3": "softmax",
        "Dropout3": "No dropout",
        "Optimizer": "adam",
        "Metric": "accuracy",
        "Epoch": 30,
        "Batch_size": 10,
        "Eta": 1e-5,
        "L2": 1e-5,
        "Loss": "binary_crossentropy"
    }
)

In [16]:
config = wandb.config

In [17]:
ds=mnist.load_data()

In [18]:
(x_train,y_train),(x_test,y_test)=ds
print(x_train.shape,x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [19]:
# Se hace al aplanado de las imágenes
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

# Se modifica el tipo de datos a tipo flotante (para evitar conflictos)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Se normalizan los datos acorde a la intensidad de los píxeles
x_train /= 255.
x_test /= 255.

In [20]:
# Se realiza el one-hot enconding:

# Se definen el número de clases que hay en la base de datos
classes = config.Layer3

# Se realiza la conversión:
y_train = keras.utils.to_categorical(y_train,classes)
y_test = keras.utils.to_categorical(y_test,classes)

In [21]:
# Se configuran los hiperparámetros del modelo:

# Valor del Learning Rate
eta = config.Eta
eta = float(eta) # Tiene que ser un dato de tipo flotante, pues de lo contrario
                 # tiene conflicto

# Número de épocas
epochs = config.Epoch

# Tamaño del mini-batch
mini_batch= config.Batch_size

In [22]:
# Se realiza la creación del modelo a como se había definido usando Numpy:
model = Sequential([
    Dense(config.Layer1, activation = config.Activation_1, input_shape = (784,), kernel_regularizer = regularizers.L2(config.L2)),
    Dense(config.Layer2, activation = config.Activation_2, kernel_regularizer = regularizers.L2(config.L2)),
    Dense(classes, activation = config.Activation_3, kernel_regularizer = regularizers.L2(config.L2))
])
model.summary()
# El modelo es secuencial, se van uniendo neurona por neurona de forma consecutiva

/home/emmdzzz/anaconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 269,322 (1.03 MB)

 Trainable params: 269,322 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(loss=config.Loss, optimizer = config.Optimizer,
              metrics=[config.Metric])

history = model.fit(x_train, y_train,
                    batch_size = mini_batch,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[WandbMetricsLogger(log_freq=5),
                               WandbModelCheckpoint("models.keras")]
                    )

Epoch 1/30


2025-09-13 01:14:30.027647: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 188160000 exceeds 10% of free system memory.


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8815 - loss: 0.0796 - val_accuracy: 0.9678 - val_loss: 0.0329
Epoch 2/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.9713 - loss: 0.0315 - val_accuracy: 0.9695 - val_loss: 0.0317
Epoch 3/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.9770 - loss: 0.0277 - val_accuracy: 0.9732 - val_loss: 0.0301
Epoch 4/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9806 - loss: 0.0253 - val_accuracy: 0.9773 - val_loss: 0.0274
Epoch 5/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9819 - loss: 0.0242 - val_accuracy: 0.9685 - val_loss: 0.0320
Epoch 6/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9850 - loss: 0.0231 - val_accuracy: 0.9799 - val_loss: 0.0259
Epoch 7/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9855 - loss: 0.0227 - val_accuracy: 0.9786 - val_loss: 0.0263
Epoch 8/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9869 - loss: 0.0220 - val

In [24]:
wandb.finish()

batch/accuracy,▁▅▅▇▆▆▆▇▇▇██▇▇▇▇▇██▇████████▇███████████
batch/batch_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▆▆▇▇▇▇▇██████████████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▂▄▆▁▇▆▅▆▆▆▅▇▅▆▅▆▇▆▆▇▆▄█▇▆▇▆▇█
epoch/val_loss,█▇▆▄▇▃▃▄▃▄▃▃▃▄▂▄▃▂▃▃▃▃▅▂▂▃▂▃▂▁
batch/accuracy,0.98873
